In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from tqdm import tqdm
PUNCATUATION = '''!\"#$%&\'()*+, -./:;<=>?@[\]^_`{|}~'''
MAX_LEN = 500

In [2]:
def preProcess(sent: str):
    sent = sent.strip('\"')
    sent = word_tokenize(sent)
    for i in PUNCATUATION:
        if i in sent:
            sent = list(filter(i.__ne__, sent))
    if len(sent) == 0:
        sent.append(' ')
    return ' '.join(sent)

def compare_token(sent_a_list, sent_b_list):
    result_list = []
    for sent_a, sent_b in zip(sent_a_list, sent_b_list):
        compare_result = []
        a_tokens = sent_a.split(sep=' ')
        b_tokens = sent_b.split(sep=' ')
        a_idx = 0
        b_idx = 0
        while a_idx < len(a_tokens):
            if b_idx == len(b_tokens):
                a_idx += 1
                compare_result.append(0)
                continue
            if a_tokens[a_idx] != b_tokens[b_idx]:
                a_idx += 1
                compare_result.append(0)
            elif a_tokens[a_idx] == b_tokens[b_idx]:
                a_idx += 1
                b_idx += 1
                compare_result.append(1)
        assert len(a_tokens) == len(compare_result), "ERROR, Len of tokens not equal to len of result!"
        result_list.append(compare_result)
    return result_list

In [3]:
BERT_BASE_CASED = 'bert-base-cased'

In [4]:
dataFrame = pd.read_csv('./Batch_answers - train_data (no-blank).csv')
dataFrame.drop(labels=['Unnamed: 6', 'total no.: 7987'], axis=1 ,inplace=True) # 將不必要的drop掉
dataFrame.rename({"q\'":"Q", "r\'":"R"}, axis=1, inplace=True)

In [5]:
dataFrame.sample(6) # 看六個row 來確認資料屬性

,id,q,r,s,Q,R
6598,1757,"""Here 's an analogy Marc9000 : Ridiculously Ab...","""I am not sure Marc will show up here much . I...",AGREE,"""Here 's an analogy Marc9000 : Ridiculously Ab...","""I am not sure Marc will show up here much . I..."
3056,805,"""So you may want us to believe that abandoning...","""I spoke no lies . I can substantiate my posit...",DISAGREE,"""as abandoning Gods truths never leads to good...","""I can substantiate my position WITH POLLS tha..."
20759,5461,"""In the vanishingly small likelihood of the ev...","""That 's why the Second Amendment protects the...",DISAGREE,"""your handgun is not going to face them down o...","""That 's why the Second Amendment protects the..."
18657,4975,"""Ok , this does n't address my point which is ...","""Well , you know what happens when they try to...",AGREE,"""this does n't address my point which is that ...","""you know what happens when they try to elimin..."
11063,2942,"""So Simone , if the shooter had instead driven...","""I think the correct term is Priuseses""",DISAGREE,"""if the shooter had instead driven his Toyota ...","""I think the correct term is Priuseses"""
38128,9956,"""HUNTSVILLE , AL Â– Jesus Christ , son of God ...","""Christ Kills Two , Injures Seven In Abortion-...",DISAGREE,"""Jesus Christ killed two wounded seven others ...","""Christ Kills Two , Injures Seven In Abortion-..."


In [6]:
print("Preprocessing q ...")
dataFrame['q'] = dataFrame['q'].map(preProcess)
print("Preprocessing r ...")
dataFrame['r'] = dataFrame['r'].map(preProcess)
print("Preprocessing Q ...")
dataFrame['Q'] = dataFrame['Q'].map(preProcess)
print("Preprocessing R ...")
dataFrame['R'] = dataFrame['R'].map(preProcess)

Preprocessing q ...
Preprocessing r ...
Preprocessing Q ...
Preprocessing R ...


In [7]:
dataFrame['com_q'] = compare_token(dataFrame['q'], dataFrame['Q'])
dataFrame['com_r'] = compare_token(dataFrame['r'], dataFrame['R'])

In [10]:
dataFrame['s'].value_counts()

DISAGREE    31428
AGREE        6918
Name: s, dtype: int64

In [11]:
dataFrame.to_csv('PreprocessFullData.csv', index=False)